In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [7]:
from dotenv import load_dotenv
load_dotenv()
key=os.getenv("OPENAI_API_KEY")

In [8]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.7)

In [13]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [10]:
with open("e:\\Girish Documents\\Study\\Data Science\\DataScience_GenAI_Study\\GenAI_MCQ-Generator_Project\\Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [11]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [15]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [16]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [17]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"], 
    template=TEMPLATE2
    )

quiz_review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review",verbose=True)

In [18]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, quiz_review_chain], 
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    output_variables=["quiz", "review"], 
    verbose=True)


In [20]:
PATH="E:\Girish Documents\Study\Data Science\DataScience_GenAI_Study\GenAI_MCQ-Generator_Project\data.txt"

In [21]:
print(PATH)

E:\Girish Documents\Study\Data Science\DataScience_GenAI_Study\GenAI_MCQ-Generator_Project\data.txt


In [22]:
with open(PATH,"r") as file:
    TEXT = file.read()

In [23]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

In [24]:
TEXT
NUMBER=5
SUBJECT="AI"
TONE="Simple",
RESPONSE_JSON=RESPONSE_JSON

In [25]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [27]:
with  get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {

        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)

        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large 

In [28]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1379
Prompt Tokens:954
Completion Tokens:425
Total Cost:0.002281


In [29]:
response

{'text': 'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:\n\nSystems, architecture and infrastructure for Generative AI\nMachine learning and Modeling using LLMs and Diffusion models\nLarge Language Models and its applications\nMulti-modal Generative AI and its applications\nGen AI b

In [30]:
quiz=response.get("quiz")

In [31]:
quiz=json.loads(quiz)

In [32]:
quiz

{'1': {'no': '1',
  'mcq': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'options': {'a': 'Building and deploying AI systems',
   'b': 'Data collection and processing',
   'c': 'Developing infrastructure',
   'd': 'Ethics in AI'},
  'correct': 'a'},
 '2': {'no': '2',
  'mcq': 'Which of the following is NOT a topic of interest in the workshop?',
  'options': {'a': 'Systems and infrastructure for Generative AI',
   'b': 'Machine learning and Modeling using LLMs',
   'c': 'Responsible use of Gen AI',
   'd': 'Blockchain technology'},
  'correct': 'd'},
 '3': {'no': '3',
  'mcq': 'What does Gen AI stand for?',
  'options': {'a': 'General Artificial Intelligence',
   'b': 'Generative Algorithmic Intelligence',
   'c': 'Generation of Artificial Intelligence',
   'd': 'Generalized AI'},
  'correct': 'b'},
 '4': {'no': '4',
  'mcq': 'What is the purpose of evaluating Language and Diffusion based models?',
  'options': {'a': 'To ensure fairness and transparency',
   'b

In [33]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [34]:
quiz_table_data

[{'MCQ': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a: Building and deploying AI systems | b: Data collection and processing | c: Developing infrastructure | d: Ethics in AI',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a topic of interest in the workshop?',
  'Choices': 'a: Systems and infrastructure for Generative AI | b: Machine learning and Modeling using LLMs | c: Responsible use of Gen AI | d: Blockchain technology',
  'Correct': 'd'},
 {'MCQ': 'What does Gen AI stand for?',
  'Choices': 'a: General Artificial Intelligence | b: Generative Algorithmic Intelligence | c: Generation of Artificial Intelligence | d: Generalized AI',
  'Correct': 'b'},
 {'MCQ': 'What is the purpose of evaluating Language and Diffusion based models?',
  'Choices': 'a: To ensure fairness and transparency | b: To improve the accuracy of the models | c: To analyze the effectiveness of Gen AI | d: To develop large language models',
  'Correct': 'c'},
 {

In [35]:
df=pd.DataFrame(quiz_table_data)

In [36]:
df.to_csv("genai.csv",index=False)

In [37]:
LOG_FILE="{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [38]:
LOG_FILE

"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [39]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'01_14_2024_18_49_46'